# Model Training for Traffic Object Detection

## Overview
This notebook trains a YOLOv5 model to detect vehicles and pedestrians from Atlanta traffic camera footage.

## Dataset Summary
- **Source**: ATL-0610 camera at 10th Street and Monroe Drive
- **Annotated frames**: 29 frames from video ATL-0610_20250609_131130
- **Classes**: 2 (vehicle, pedestrian)
- **Annotations**: YOLO format exported from CVAT
- **Location**: `/home/trauco/data-science-sad/annotations/yolo/`

## Training Objectives
1. Train initial YOLOv5s model (smallest variant for quick iteration)
2. Validate model performance on annotated data
3. Establish baseline metrics for future improvements

## Expected Outputs
- Trained model weights: `runs/train/exp/weights/best.pt`
- Training metrics and loss curves
- Validation results on test split

## Setup and Environment Verification

Initialize the training environment by importing required libraries, checking GPU availability, and verifying that annotation files from CVAT export are present in the expected directory structure.

## Environment Update Required

**Missing packages: PyTorch and Ultralytics**

### Action Items for Reproducibility Framework
1. Update `environment.yml` to include PyTorch dependencies
2. Update `environment.yml` to include Ultralytics 
3. Document GPU requirements (NVIDIA RTX A5500, CUDA 12.0)
4. Add installation verification steps to setup documentation

### Installation Instructions
```bash
# activate environment
conda activate traffic-vision

# install pytorch with cuda support
conda install pytorch torchvision torchaudio pytorch-cuda=11.8 -c pytorch -c nvidia

# install ultralytics
pip install ultralytics

# verify installations
python -c "import torch; print(f'PyTorch: {torch.__version__}'); print(f'CUDA: {torch.cuda.is_available()}')"
python -c "import ultralytics; print(f'Ultralytics: {ultralytics.__version__}')"

In [1]:
# imports
import os
import yaml
from pathlib import Path
import torch
from ultralytics import YOLO
import pandas as pd
import matplotlib.pyplot as plt

# gpu check
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")
    
# paths
PROJECT_ROOT = Path("/home/trauco/data-science-sad")
ANNOTATIONS_PATH = PROJECT_ROOT / "annotations" / "yolo"

# verify files
print(f"\nChecking annotation files in: {ANNOTATIONS_PATH}")
print(f"obj.names exists: {(ANNOTATIONS_PATH / 'obj.names').exists()}")
print(f"train.txt exists: {(ANNOTATIONS_PATH / 'train.txt').exists()}")
print(f"obj_train_data exists: {(ANNOTATIONS_PATH / 'obj_train_data').is_dir()}")

PyTorch version: 2.3.1+cu121
CUDA available: True
CUDA device: NVIDIA RTX A5500

Checking annotation files in: /home/trauco/data-science-sad/annotations/yolo
obj.names exists: True
train.txt exists: True
obj_train_data exists: True


## Inspect Annotation Data Structure

This cell examines the YOLO format annotations exported from CVAT to understand:
- Class names defined in obj.names
- Number of annotated images in train.txt
- YOLO annotation format (class_id x_center y_center width height)

In [2]:
# load annotations
with open(ANNOTATIONS_PATH / 'obj.names', 'r') as f:
    classes = f.read().strip().split('\n')
print(f"Classes: {classes}")

# check train list
with open(ANNOTATIONS_PATH / 'train.txt', 'r') as f:
    train_images = f.readlines()
print(f"\nTraining images: {len(train_images)}")
print(f"First image: {train_images[0].strip()}")

# sample annotation
sample_image = Path(train_images[0].strip())
annotation_file = sample_image.with_suffix('.txt')
if annotation_file.exists():
    with open(annotation_file, 'r') as f:
        annotations = f.readlines()
    print(f"\nSample annotations ({annotation_file.name}):")
    for ann in annotations[:5]:  # first 5
        print(f"  {ann.strip()}")

Classes: ['vehicle', 'pedestrian']

Training images: 129
First image: data/obj_train_data/frame_131130_000.jpg


## Fix Annotation File Paths

The train.txt file contains relative paths that need to be converted to absolute paths for YOLO training.

In [3]:
# check current path
sample_relative = train_images[0].strip()
print(f"Relative path in train.txt: {sample_relative}")

# build absolute path
sample_absolute = ANNOTATIONS_PATH / sample_relative
print(f"Absolute path: {sample_absolute}")
print(f"Image exists: {sample_absolute.exists()}")

# check annotation
annotation_absolute = sample_absolute.with_suffix('.txt')
print(f"Annotation exists: {annotation_absolute.exists()}")

# count actual files
if (ANNOTATIONS_PATH / "obj_train_data").exists():
    images = list((ANNOTATIONS_PATH / "obj_train_data").glob("*.jpg"))
    annotations = list((ANNOTATIONS_PATH / "obj_train_data").glob("*.txt"))
    print(f"\nActual files in obj_train_data:")
    print(f"  Images: {len(images)}")
    print(f"  Annotations: {len(annotations)}")

Relative path in train.txt: data/obj_train_data/frame_131130_000.jpg
Absolute path: /home/trauco/data-science-sad/annotations/yolo/data/obj_train_data/frame_131130_000.jpg
Image exists: False
Annotation exists: False

Actual files in obj_train_data:
  Images: 129
  Annotations: 129


## Locate Missing Images

The CVAT export created annotation files but didn't include the images. We need to find where the original images are and prepare the dataset structure.

In [4]:
# check annotation structure
obj_train_path = ANNOTATIONS_PATH / "obj_train_data"
print(f"Contents of {obj_train_path}:")
files = list(obj_train_path.glob("*"))[:5]  # first 5
for f in files:
    print(f"  {f.name}")

# find original images
original_images = PROJECT_ROOT / "annotation_sample"
print(f"\nOriginal images location: {original_images}")
print(f"Exists: {original_images.exists()}")
if original_images.exists():
    orig_files = list(original_images.glob("*.jpg"))
    print(f"Image count: {len(orig_files)}")
    if orig_files:
        print(f"First image: {orig_files[0].name}")

Contents of /home/trauco/data-science-sad/annotations/yolo/obj_train_data:
  frame_131130_056.txt
  frame_131130_046.txt
  frame_131130_031.jpg
  frame_131130_063.txt
  frame_131130_005.jpg

Original images location: /home/trauco/data-science-sad/annotation_sample
Exists: True
Image count: 129
First image: frame_131130_031.jpg


## Prepare Dataset Structure

The images need to be copied to the YOLO dataset directory to match the annotation files. This cell copies the annotated images to the correct location.

In [5]:
# copy images to match annotations
import shutil

source_dir = PROJECT_ROOT / "annotation_sample"
dest_dir = ANNOTATIONS_PATH / "obj_train_data"

# copy images
copied = 0
for txt_file in dest_dir.glob("*.txt"):
    img_name = txt_file.stem + ".jpg"
    source_img = source_dir / img_name
    dest_img = dest_dir / img_name
    
    if source_img.exists() and not dest_img.exists():
        shutil.copy(source_img, dest_img)
        copied += 1

print(f"Copied {copied} images")

# verify
images_after = list(dest_dir.glob("*.jpg"))
annotations_after = list(dest_dir.glob("*.txt"))
print(f"\nDataset ready:")
print(f"  Images: {len(images_after)}")
print(f"  Annotations: {len(annotations_after)}")

Copied 0 images

Dataset ready:
  Images: 129
  Annotations: 129


## Create YOLO Dataset Configuration

This cell creates a YAML configuration file that tells YOLO where to find the training data and what classes to detect.

In [6]:
# create dataset yaml
dataset_config = {
    'path': str(ANNOTATIONS_PATH),
    'train': 'obj_train_data',
    'val': 'obj_train_data',  # same for now
    'names': {
        0: 'vehicle',
        1: 'pedestrian'
    }
}

# save yaml
yaml_path = PROJECT_ROOT / 'traffic_dataset.yaml'
with open(yaml_path, 'w') as f:
    yaml.dump(dataset_config, f, default_flow_style=False)

print(f"Created dataset config: {yaml_path}")
print("\nContents:")
print(yaml.dump(dataset_config, default_flow_style=False))

Created dataset config: /home/trauco/data-science-sad/traffic_dataset.yaml

Contents:
names:
  0: vehicle
  1: pedestrian
path: /home/trauco/data-science-sad/annotations/yolo
train: obj_train_data
val: obj_train_data



## Initialize and Train YOLO Model

This cell initializes a YOLOv8 nano model (smallest and fastest) and starts training on the annotated traffic data. Using a small model is ideal for initial testing with limited data.

In [7]:
# initialize model
model = YOLO('yolov8n.pt')  # nano model

# training parameters
epochs = 50  # quick training
batch_size = 16  # small batch
imgsz = 640  # standard size

print(f"Starting training:")
print(f"  Model: YOLOv8n")
print(f"  Epochs: {epochs}")
print(f"  Batch size: {batch_size}")
print(f"  Image size: {imgsz}")
print(f"  Dataset: {yaml_path}")

# train
results = model.train(
    data=str(yaml_path),
    epochs=epochs,
    batch=batch_size,
    imgsz=imgsz,
    device=0,  # gpu
    project=str(PROJECT_ROOT / 'runs'),
    name='traffic_v1',
    exist_ok=True
)

Starting training:
  Model: YOLOv8n
  Epochs: 50
  Batch size: 16
  Image size: 640
  Dataset: /home/trauco/data-science-sad/traffic_dataset.yaml
Ultralytics 8.3.154 🚀 Python-3.11.13 torch-2.3.1+cu121 CUDA:0 (NVIDIA RTX A5500, 24090MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/home/trauco/data-science-sad/traffic_dataset.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi

train: Scanning /home/tra

train: New cache created: /home/trauco/data-science-sad/annotations/yolo/obj_train_data.cache


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 186.5±94.9 MB/s, size: 38.4 KB)


val: Scanning /home/trauc


Plotting labels to /home/trauco/data-science-sad/runs/traffic_v1/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to /home/trauco/data-science-sad/runs/traffic_v1
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/50      2.01G   


RuntimeError: Caught RuntimeError in pin memory thread for device 0.
Original Traceback (most recent call last):
  File "/home/trauco/miniconda/envs/traffic-vision/lib/python3.11/site-packages/torch/utils/data/_utils/pin_memory.py", line 37, in do_one_step
    data = pin_memory(data, device)
           ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/trauco/miniconda/envs/traffic-vision/lib/python3.11/site-packages/torch/utils/data/_utils/pin_memory.py", line 68, in pin_memory
    clone.update({k: pin_memory(sample, device) for k, sample in data.items()})
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/trauco/miniconda/envs/traffic-vision/lib/python3.11/site-packages/torch/utils/data/_utils/pin_memory.py", line 68, in <dictcomp>
    clone.update({k: pin_memory(sample, device) for k, sample in data.items()})
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/trauco/miniconda/envs/traffic-vision/lib/python3.11/site-packages/torch/utils/data/_

## Fix Torchvision Compatibility

The installed torchvision version is incompatible with PyTorch. This needs to be reinstalled to match the PyTorch version.

In [8]:
# check versions
import torch
import torchvision

print(f"PyTorch version: {torch.__version__}")
print(f"Torchvision version: {torchvision.__version__}")
print(f"CUDA version: {torch.version.cuda}")

# fix command
print("\nRun this in terminal to fix:")
print("conda activate traffic-vision")
print("pip uninstall torchvision -y")
print("pip install torchvision==0.18.1")

AttributeError: partially initialized module 'torchvision' has no attribute 'extension' (most likely due to a circular import)

## Restart Kernel Required

The torchvision module has import errors. The kernel needs to be restarted and dependencies checked.

### Steps:
1. Restart the Jupyter kernel (Kernel → Restart)
2. Check installed versions
3. Fix any version mismatches

In [9]:
# check pytorch only
import sys
print(f"Python: {sys.version}")

try:
    import torch
    print(f"PyTorch: {torch.__version__}")
except Exception as e:
    print(f"PyTorch error: {e}")

try:
    import torchvision
    print(f"Torchvision: {torchvision.__version__}")
except Exception as e:
    print(f"Torchvision error: {e}")

Python: 3.11.13 (main, Jun  5 2025, 13:12:00) [GCC 11.2.0]
PyTorch: 2.3.1.post100
Torchvision error: partially initialized module 'torchvision' has no attribute 'extension' (most likely due to a circular import)


## Fix Torchvision Installation

Torchvision has a broken installation. It needs to be completely removed and reinstalled with the correct version for PyTorch 2.3.1.

## Torchvision Installation Fixed

The torchvision module was reinstalled with the correct version compatible with PyTorch 2.3.1.

### Commands Executed:
```bash
# activate environment
conda activate traffic-vision

# completely remove torchvision
pip uninstall torchvision -y
conda remove torchvision -y

# install compatible version
pip install torchvision==0.18.1

# test it
python -c "import torchvision; print(f'Torchvision: {torchvision.__version__}')"